In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img
from tensorflow.keras.applications import vgg19
from tensorflow.keras import backend as K

# Load and preprocess the image
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = load_img(img_path, target_size=target_size)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)  # Preprocess the image for VGG19
    return img

# Deprocess the image to view it
def deprocess_image(x):
    x = x.reshape((224, 224, 3))
    x = x[:, :, ::-1]  # BGR to RGB
    x += 103.939, 116.779, 123.68  # Add the mean pixel values
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# Load the content and style images
content_img_path = './ColorizationImages/color/5.jpg'  # Đường dẫn đến ảnh nội dung
style_img_path = 'style_image.jpg'      # Đường dẫn đến ảnh phong cách

content_image = load_and_preprocess_image(content_img_path)
style_image = load_and_preprocess_image(style_img_path)

# Define the model
model = vgg19.VGG19(weights='imagenet', include_top=False)

# Define the layers to use for content and style
content_layer = 'block5_conv2'
style_layers = [
    'block1_conv1',
    'block2_conv1',
    'block3_conv1',
    'block4_conv1',
    'block5_conv1'
]

# Get the output of the model for the specified layers
outputs = [model.get_layer(layer).output for layer in style_layers]
outputs.append(model.get_layer(content_layer).output)
model = tf.keras.models.Model(inputs=model.input, outputs=outputs)

# Define the loss functions
def get_content_loss(base_content, target):
    return K.sum(K.square(base_content - target))

def get_style_loss(base_style, target):
    S = gram_matrix(base_style)
    C = gram_matrix(target)
    return K.sum(K.square(S - C))

def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# Combine the losses
def total_loss(outputs, content_image, style_image, content_weight=1e4, style_weight=1e-2):
    style_losses = [get_style_loss(output, style_image) for output in outputs[:-1]]
    content_loss = get_content_loss(outputs[-1], content_image)
    return content_weight * content_loss + style_weight * K.sum(style_losses)

# Prepare the generated image
generated_image = K.variable(content_image)

# Define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.02)

# Create a Keras function to compute the gradients
@tf.function
def compute_loss_and_grads():
    with tf.GradientTape() as tape:
        outputs = model(generated_image)
        loss = total_loss(outputs, content_image, style_image)
    grads = tape.gradient(loss, generated_image)
    return loss, grads

# Training loop
iterations = 100  # Số vòng lặp
for i in range(iterations):
    loss, grads = compute_loss_and_grads()
    optimizer.apply_gradients([(grads, generated_image)])
    
    if i % 10 == 0:
        print(f"Iteration {i}, Loss: {loss.numpy()}")
        img = deprocess_image(generated_image.numpy())
        plt.imshow(img)
        plt.axis('off')
        plt.show()

# Save the final generated image
final_img = deprocess_image(generated_image.numpy())
save_img('output_image.jpg', final_img)
